# Mリーグの牌譜データを取得して保存する
 - 2021/05/30：動作確認
 - オフィシャルサポーターの特典の牌譜ビューアからデータを取得
   - オフィシャルサポーターになる必要があります
 - ChromeでSeleniumを動かせる環境が必要です
   - 動作確認したChromeのバージョンは「90.0.4430.93」
 - **データ取得はサーバへの負荷のかからない範囲で行うようにしてください**
 - **ページ構造上, 次のシーズンが始まると一部要修正**

## Import

In [33]:
import time
import json
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

## Version

In [34]:
webdriver.__version__

'3.14.1'

## Setting

In [35]:
pd.options.display.max_rows = 200

# アカウント情報の読込
account = pd.read_csv("../account.txt", header=None, sep=',')
MAIL = account.loc[0,0]
PASS = account.loc[0,1]

## Function

In [44]:
def login_fun(_mail, _pass):
    # ログインページにアクセス
    driver.get("https://m-league.jp/supporter/login")
    
    # Mail, Passを入力
    id_input = driver.find_element_by_id("login_mail")
    id_input.send_keys(_mail)
    pass_input = driver.find_element_by_id("login_password")
    pass_input.send_keys(_pass)
    
    # ログインボタンをクリック
    login_button = driver.find_elements_by_name("login")
    login_button[1].click()

def get_match_schedule_fun(season):
    
    match_schedule_list = []
    
    # 試合日程ページへアクセス
    if season == 2020:
        url = "https://m-league.jp/games"
    else:
        url = f"https://m-league.jp/games/{season}-season"
    driver.get(url)
    
    # 20xxシーズンは20xx+1まで開催
    for y in [season, season+1]:
        for m in range(1, 12+1, 1):
                for d in range(1, 31+1, 1):
                    ymd = str(y) + str(m).zfill(2) + str(d).zfill(2)
                    if len(driver.find_elements_by_xpath(f".//div[@id='js-modal-key{ymd}']")) == 1:
                        match_schedule_list += [ymd]
    
    return match_schedule_list

def make_score_data_fun(html):
    # htmlをパースする
    soup = BeautifulSoup(html, "html.parser")
    
    # 必要な個所だけ抽出
    start_mark = "UMP_PLAYER.init(true, true, \'"
    end_mark = "\', autoplay);"
    start_position = str(soup).find(start_mark) + len(start_mark)
    end_position = str(soup).find(end_mark)
    score = str(soup)[start_position:end_position]
    
    # DataFrameに変換
    dat = pd.DataFrame(json.loads(score))
    
    return dat

def make_dat_2020_fun(match_schedule_list):
    driver.get("https://m-league.jp/games")

    dat_list = []
    button_count = 0
    # 試合日程は時系列にソート済み想定
    for m_s in match_schedule_list:
        # 月選択
        tmp = driver.find_element_by_xpath(f".//a[@href='/games/?mly={m_s[0:4]}&mlm={int(m_s[4:6])}#schedule']")
        tmp.click()
        # 日程選択
        tmp = driver.find_element_by_xpath(f".//li[@data-target='key{m_s}']")
        tmp.click()
        
        time.sleep(1)
        # 牌譜ビューアにアクセスするボタン取得
        button = driver.find_elements_by_xpath(".//button[@class='c-button -primary -small']")
        button[button_count].click()

        ### 第1試合
        # 制御するタブの移動
        driver.switch_to.window(driver.window_handles[1])
        # 文字コードをUTF-8に変換
        html = driver.page_source.encode('utf-8')
        # DataFrameに変換
        dat = make_score_data_fun(html)
        dat_list += [dat]
        # 牌譜ビューアを閉じる
        driver.close()
        # 制御するタブの移動
        driver.switch_to.window(driver.window_handles[0])

        ### 第2試合
        button[button_count+1].click()
        # 制御するタブの移動
        driver.switch_to.window(driver.window_handles[1])
        # 文字コードをUTF-8に変換
        html = driver.page_source.encode('utf-8')
        # DataFrameに変換
        dat = make_score_data_fun(html)
        dat_list += [dat]
        # 牌譜ビューアを閉じる
        driver.close()
        # 制御するタブの移動
        driver.switch_to.window(driver.window_handles[0])

        # ページのリセット
        driver.get("https://m-league.jp/games")
        # button_countを次の日程にズラす
        button_count += 2

        # アクセス制限
        time.sleep(10)
        print("Get: " + m_s)
        
    # データ縦積み
    dat_all = pd.concat(dat_list).reset_index(drop=True)
    
    return dat_all

## Get Data
### ブラウザの起動

In [37]:
# ブラウザのオプション
options = Options()

# Headless
options.headless = False

# ブラウザ起動
driver = webdriver.Chrome(executable_path="chromedriver.exe", options=options)

### オフィシャルサポーターとしてログイン

In [38]:
login_fun(MAIL, PASS)

### 試合日程の取得
　下記範囲で探索してwebページと一致するものを取得, 2018は78日程, 2019は108日程.
- 年は2018～2021
- 月は1～12
- 日は1～31

In [39]:
# 2020シーズン
match_schedule_list_2020 = get_match_schedule_fun(2020)
match_schedule_list_2020.sort(reverse=False)
len(match_schedule_list_2020)

108

In [40]:
match_schedule_list_2020[0:5]

['20201005', '20201006', '20201008', '20201009', '20201012']

In [41]:
# 2019シーズン
match_schedule_list_2019 = get_match_schedule_fun(2019)
match_schedule_list_2019.sort(reverse=False)
len(match_schedule_list_2019)

108

In [42]:
# 2018シーズン
match_schedule_list_2018 = get_match_schedule_fun(2018)
match_schedule_list_2018.sort(reverse=False)
len(match_schedule_list_2018)

78

### 牌譜データ読込

In [45]:
dat_2020 = make_dat_2020_fun(match_schedule_list_2020)

Get: 20201005
Get: 20201006
Get: 20201008
Get: 20201009
Get: 20201012
Get: 20201013
Get: 20201015
Get: 20201016
Get: 20201019
Get: 20201020
Get: 20201022
Get: 20201023
Get: 20201026
Get: 20201027
Get: 20201029
Get: 20201030
Get: 20201102
Get: 20201103
Get: 20201105
Get: 20201106
Get: 20201109
Get: 20201110
Get: 20201112
Get: 20201113
Get: 20201116
Get: 20201117
Get: 20201119
Get: 20201120
Get: 20201123
Get: 20201124
Get: 20201126
Get: 20201127
Get: 20201130
Get: 20201201
Get: 20201203
Get: 20201204
Get: 20201207
Get: 20201208
Get: 20201210
Get: 20201211
Get: 20201214
Get: 20201215
Get: 20201217
Get: 20201218
Get: 20201221
Get: 20201222
Get: 20201224
Get: 20201225
Get: 20210102
Get: 20210103
Get: 20210104
Get: 20210105
Get: 20210107
Get: 20210108
Get: 20210111
Get: 20210112
Get: 20210114
Get: 20210115
Get: 20210118
Get: 20210119
Get: 20210121
Get: 20210122
Get: 20210125
Get: 20210126
Get: 20210128
Get: 20210129
Get: 20210201
Get: 20210202
Get: 20210204
Get: 20210205
Get: 20210208
Get: 2

### データ確認

In [46]:
dat_2020.head(200)

,time,id,cmd,args
0,2020-10-05 19:11:00,0,area,[-]
1,2020-10-05 19:11:00,1,player,"[A0, 高宮 まり, user, T003]"
2,2020-10-05 19:11:00,2,player,"[B0, 瀬戸熊 直樹, user, T006]"
3,2020-10-05 19:11:00,3,player,"[C0, 内川 幸太郎, user, T008]"
4,2020-10-05 19:11:00,4,player,"[D0, 二階堂 亜樹, user, T002]"
5,2020-10-05 19:11:00,5,gamestart,[id=L001_S007_0001_01A]
6,2020-10-05 19:11:00,0,kyokustart,"[0, A0, 0, 0, 1z, 1z, 2z, 3z, 4z]"
7,2020-10-05 19:11:00,1,point,"[A0, =25000]"
8,2020-10-05 19:11:00,2,point,"[B0, =25000]"
9,2020-10-05 19:11:00,3,point,"[C0, =25000]"


### CSV出力

In [47]:
dat_2020.to_csv("../data/M_League_2020.csv", header=False, index=False)